In [9]:
%matplotlib widget
import pickle
from Particle import Particle
import matplotlib.pyplot as plt
import numpy as np

In [10]:
filename = 'off_lat'
infile = open(filename,'rb')
cluster = pickle.load(infile)

## Measure the radical distance

In [11]:
np.sqrt(np.sum(np.power(np.array((2,3))-np.array((1,1)),2),axis=-1))

2.23606797749979

In [12]:
def dist(a,b):
    return np.sqrt(np.sum(np.power(np.array(a)-np.array(b),2),axis=-1))

In [44]:
def radical_fractal_dim(cluster, center, delta_r):
    size = len(cluster)
    rads = np.array([dist(particle.loc, center) for particle in cluster])
    r_max = np.max(rads, axis=-1)
    r_ranges = np.arange(0,r_max+1,delta_r)
    ret = np.zeros_like(r_ranges)
    for i in range(len(r_ranges)):
        ret[i] = np.sum(rads < r_ranges[i])
    return np.array([ret,r_ranges])

In [47]:
svn = radical_fractal_dim(cluster, (250,250), 1)

array([0., 0.])

In [51]:
plt.figure()
plt.scatter(svn[0], svn[1])
p = np.polyfit(svn[0], svn[1], 1)
print(p)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[0.39329893 0.20903848]


# Convert into on lattice measurement

In [14]:
def to_lattice(cluster, shape, sigma=1):
    lattice = np.zeros(shape)
    for particle in cluster:
        i = int(particle.loc[0]/sigma)
        j = int(particle.loc[1]/sigma)
        lattice[i, j] = 1
    return lattice

In [15]:
clus_lat = to_lattice(cluster, shape=(500,500))

In [16]:
plt.figure()
plt.imshow(clus_lat, cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
def calculate_fractal_dim(dla_lattice):
    leni, lenj = dla_lattice.shape
    LMIDi = int(leni / 2)
    LMIDj = int(lenj / 2)
    LMID = min(LMIDi, LMIDj)  # let's ensure quadratic boxes which fit on DLAlattice
    size_vs_num = np.zeros((LMID, 2))
    # loop over s which gives b=2*s+1
    for size in range(LMID):
        num = 0
        # loop over lattice
        for x in range(LMIDi - size, LMIDi + size + 1):
            for y in range(LMIDj - size, LMIDj + size + 1):
                num += dla_lattice[x, y]
        if num > 0:
            size_vs_num[size] = np.array([np.log(2 * size + 1), np.log(num)])
    return size_vs_num

In [18]:
size_vs_num = calculate_fractal_dim(mat)

In [19]:
plt.figure()
plt.scatter(size_vs_num[:, 0], size_vs_num[:, 1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
idx = np.where(size_vs_num[:, 0] > 6)[0][1]
idx_e = np.where(size_vs_num[:, 0] < 5)[0][-1]
print(idx_e,idx)
p = np.polyfit(size_vs_num[idx_e:idx, 0], size_vs_num[idx_e:idx, 1], 1)
print(p)

73 203
[ 2.49229068 -3.04109493]
